# The Monty Hall Problem
## *Would You Switch?*

---

It's the 1970s. You're a contestant on *Let's Make a Deal*, one of the most popular game shows in America. The host, Monty Hall, stands in front of three closed doors. The studio audience is buzzing.

Behind one door is a brand-new car. Behind the other two — goats.

You pick a door. Say, Door #1. But before you can open it, Monty — who knows *exactly* what's behind every door — walks over to another door and swings it open. A goat stares back at you. The audience laughs.

Now only two doors remain: the one you picked, and one other. Monty turns to you with a smile and asks:

> ***"Would you like to switch?"***

Your gut says it doesn't matter — it's 50/50 now, right? Two doors, one car.

But what if your gut is wrong?

## Let's Settle This With an Experiment

Instead of trying to reason through the answer, let's do something a scientist would do — run the experiment.

We'll write a program that plays the game for us, hundreds of times. One version always stays with the original door. The other always switches. If we play enough rounds, the truth should reveal itself in the numbers.

In [ ]:
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sns.set_theme(style="ticks", font_scale=1.2)

def run_monty_hall(num_trials: int, switch_door: bool) -> int:
    """Play the Monty Hall game num_trials times and return total wins."""
    wins = 0
    doors = [0, 1, 2]

    for _ in range(num_trials):
        prize_door   = random.choice(doors)
        player_pick  = random.choice(doors)

        # Monty opens a door that is neither the prize nor the player's pick
        monty_opens  = random.choice(
            [d for d in doors if d != prize_door and d != player_pick]
        )

        if switch_door:
            final = [d for d in doors if d != player_pick and d != monty_opens][0]
        else:
            final = player_pick

        if final == prize_door:
            wins += 1

    return wins

Drag the slider to increase the number of games. Start small and watch the results jump around. Then crank it up. As the sample grows, the numbers settle toward something very specific.

*What do they converge to? Is it what you expected?*

In [ ]:
@widgets.interact(n=widgets.IntSlider(
    value=100, min=100, max=1000, step=100,
    description="# Games:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="500px"),
))
def plot_monty_hall(n):
    wins_stay   = run_monty_hall(n, switch_door=False)
    wins_switch = run_monty_hall(n, switch_door=True)

    df = pd.DataFrame({
        "Strategy": ["Always Stay", "Always Switch"],
        "Wins":     [wins_stay, wins_switch],
        "Win %":    [round(wins_stay / n * 100, 1), round(wins_switch / n * 100, 1)]
    })

    fig, ax = plt.subplots(figsize=(7, 5))
    sns.barplot(data=df, x="Strategy", y="Wins",
                hue="Strategy", dodge=False, legend=False,
                palette=["#E8575A", "#5B8FB9"],
                edgecolor="white", linewidth=1.5, width=0.5, ax=ax)

    ax.set_title(f"Monty Hall Results — {n:,} Games per Strategy",
                 fontsize=14, weight="bold", pad=12)
    ax.set_ylabel("Number of Wins")
    ax.set_xlabel("")
    ax.set_ylim(0, n * 1.1)

    for i, container in enumerate(ax.containers):
        row = df.iloc[i]
        ax.bar_label(container,
                     labels=[f"{row['Wins']} wins\n({row['Win %']}%)"],
                     fontsize=12, fontweight="bold", padding=6)

    ax.axhline(n * 1/3, color="#E8575A", linestyle="--", linewidth=1,
               label=f"Theory: Stay = 33.3% ({n//3} wins)")
    ax.axhline(n * 2/3, color="#5B8FB9", linestyle="--", linewidth=1,
               label=f"Theory: Switch = 66.7% ({n*2//3} wins)")
    ax.legend(fontsize=10)

    sns.despine(left=True, bottom=True)
    plt.tight_layout()
    plt.show()

    print(f"\nStay win rate:   {wins_stay/n*100:.1f}%  (theory: 33.3%)")
    print(f"Switch win rate: {wins_switch/n*100:.1f}%  (theory: 66.7%)")

interactive(children=(IntSlider(value=100, description='# Games:', layout=Layout(width='500px'), max=1000, min…

## So Why Does Switching Work?

Here's the insight that trips up nearly everyone — including mathematicians who famously argued about this in the 1990s.

When you first picked a door, you had a 1-in-3 chance of being right. That means there was a 2-in-3 chance the car was behind one of the *other* doors. Now, Monty opens a door he *knows* has a goat — he's not choosing randomly. He's giving you information. That entire 2/3 probability collapses onto the single remaining door.

Stay — you win only if your first guess was right — **1/3.** Switch — you win whenever your first guess was *wrong* — **2/3.**

Switching doesn't just help a little. It **doubles** your chances. And you didn't need to trust the math — you ran the experiment yourself, and the data told you the same story.

---

*This puzzle became famous in 1990 when Marilyn vos Savant published the answer in a magazine. Thousands of PhD holders wrote in to say she was wrong. She wasn't.*